In [19]:
from scipy.integrate import solve_ivp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Latex, Math
from scipy.optimize import least_squares
import altair as alt
from ipywidgets import interactive

In [20]:
def nomoto_zigzag(t, states, K, T_1, angle=np.deg2rad(10)):
    """
    Simulation model for heading and yaw rate using Nomoto with only K and T_1
    Also including an autopilot for ZigZag test:
    """
    
    
    # states: 
    psi = states[0]
    r = states[1]
    
    #ZigZag autopilot:
    sign=-1
    psi_pluss = ((psi>=0) and (psi<angle))
    psi_minus = ((psi>=-angle) and (psi<=0))
    
    if (
        (psi_pluss and (r>=0)) |
        (psi<-angle) |
        (psi_minus and (r>=0))
        
        
    ):
        sign=1   
    delta_=sign*angle
    
    #Nomoto equation:
    r1d_ = (K*delta_ - r)/T_1 
    d_states_dt = np.array([r,r1d_])
    
    return d_states_dt

def simulate_zigzag(K = 0.20, T_1=10.0):

    ## Simulation:
    
    angle=np.deg2rad(10)
    states_0 = np.array([0,0])
    t = np.linspace(0,60,200)
    
    sol = solve_ivp(fun = nomoto_zigzag,t_span=[t[0],t[-1]],t_eval=t, y0 = states_0, args=(K,T_1,angle))
    psi = sol.y[0,:]
    r = sol.y[1,:]
    
    ## Plotting:
    fig,ax=plt.subplots()
    ax.plot(t,np.rad2deg(psi));
    ax.grid(True)
    ax.set_title('Simulation ZigZag%0.0f/%0.0f with Nomoto model' % (np.rad2deg(angle),np.rad2deg(angle)))
    ax.set_ylabel('$\psi$ [deg]')
    ax.set_xlabel('time [s]');
    ax.set_ylim(-80,80)

In [21]:
#collapse
interactive_plot = interactive(simulate_zigzag, K=(0.01, 3.0, 0.01), T_1=(1, 15, 0.1))
output = interactive_plot.children[-1]
output.layout.height = '1250px'
interactive_plot

interactive(children=(FloatSlider(value=0.2, description='K', max=3.0, min=0.01, step=0.01), FloatSlider(value…

In [10]:
# First order Nomoto model equation
# https://martinlarsalbert.github.io/blog/simulation/dynamics/manoeuvring/system%20identification/nomoto/2020/08/31/nomoto_equation.html